In [1]:
import pandas as pd

## PATHS

In [2]:
PERIODS = 'menstrual_health_cycles.csv'
SLEEP_score = 'sleep/sleep_score.csv'
SLEEP = 'sleep/sleep-2022-03-20.json'

## periods

In [3]:
per = pd.read_csv(PERIODS)
per.drop(['id','ovulation_source', 'period_source', 'fertile_source', 'ovulation_start_date','ovulation_end_date','fertile_start_date','fertile_end_date'], axis=1, inplace=True)
per['cycle_start_date'] = pd.to_datetime(per['cycle_start_date'])
per['cycle_end_date'] = pd.to_datetime(per['cycle_end_date'])
per['period_start_date'] = pd.to_datetime(per['period_start_date'])
per['period_end_date'] = pd.to_datetime(per['period_end_date'])
per['cycle_length'] = per['cycle_end_date'] - per['cycle_start_date']
per['period_length'] = per['period_end_date'] - per['period_start_date']
per['cycle_length'] = pd.to_timedelta(per['cycle_length']).dt.days
per['period_length'] = pd.to_timedelta(per['period_length']).dt.days
per.drop([per.index[37], per.index[38], per.index[39]], inplace=True)
per.to_csv('interim_data/periods.csv', columns=['cycle_start_date','cycle_length','period_length'])

In [4]:
per['period_start_date'] = per['period_start_date'].dt.date
per['period_end_date'] = per['period_end_date'].dt.date

per.head()

,cycle_start_date,cycle_end_date,period_start_date,period_end_date,cycle_length,period_length
0,2019-05-13,2019-06-13,2019-05-13,2019-05-19,31,6
1,2019-06-14,2019-07-14,2019-06-14,2019-06-18,30,4
2,2019-07-15,2019-08-12,2019-07-15,2019-07-21,28,6
3,2019-08-13,2019-09-07,2019-08-13,2019-08-17,25,4
4,2019-09-08,2019-10-06,2019-09-08,2019-09-12,28,4


### Sleep Data

In [5]:
scores = pd.read_csv(SLEEP_score)
scores['timestamp'] = pd.to_datetime(scores['timestamp'])
scores['date'] = scores['timestamp'].dt.date
scores['wake_time'] = scores['timestamp'].dt.time
scores.drop(['sleep_log_entry_id', 'timestamp'], axis=1, inplace=True)

In [6]:
sleep_files = ['2020-05-29', '2020-06-28','2020-07-28','2020-08-27','2020-09-26','2020-10-26','2020-11-25','2020-12-25',
               '2021-01-24','2021-02-23','2021-03-25','2021-04-24','2021-05-24','2021-06-23','2021-07-23','2021-08-22',
               '2021-09-21','2021-10-21', '2021-11-20', '2021-12-20','2022-01-19', '2022-02-18', '2022-03-20']

In [7]:
supe = pd.DataFrame()
for fi in sleep_files:
    sleep = pd.read_json('sleep/sleep-' + fi + '.json')
    sleep['endTime'] = pd.to_datetime(sleep['endTime'])
    
    sleep = sleep[sleep['mainSleep']==True]
    temp = pd.DataFrame()
    
    for x in range(sleep.shape[0]):
        try:
            a = sleep['levels'][x]['summary']['deep']['minutes']
        except: 
            a = False
            #print('a', fi)
        try:
            b = sleep['levels'][x]['summary']['wake']['minutes']
        except: 
            b = False
            #print('b', fi)
        try:
            c = sleep['levels'][x]['summary']['light']['minutes']
        except: 
            c = False
            #print('c', fi)
        try:
            d = sleep['levels'][x]['summary']['rem']['minutes']
        except: 
            d= False
            #print('d', fi)
        
        temp[x] = a, b, c, d

    temp = temp.T
    temp.columns = ['deep','wake', 'light','rem']

    sleep = sleep[['endTime', 'minutesAsleep']]
    sleep = pd.concat([sleep, temp], axis=1)
    sleep['date'] = sleep['endTime'].dt.date
    sleep['wake_time'] = sleep['endTime'].dt.time
    sleep.drop('endTime', axis=1, inplace=True)
    sleep = sleep[['date','minutesAsleep','deep','light','rem','wake', 'wake_time']]

    sleep['in_bed_time (mins)'] = sleep['minutesAsleep'] + sleep['wake']
    
    supe = pd.concat([supe, sleep], axis=0)

In [8]:
'''a = supe[supe.duplicated(['date'], keep='last')==True]
b = supe[supe.duplicated(['date'])==True]
dup = pd.concat([a, b], axis = 0)
dup.sort_values('date')''';

In [9]:
supe.drop_duplicates('date', inplace=True)
supe.set_index('date', inplace=True)
scores = scores.join(supe, on='date', how='left', lsuffix='s')
scores.set_index('date', inplace=True)
scores.to_csv('interim_data/sleep.csv')

In [10]:
print(supe.shape[0])
supe.dropna(inplace=True)
print(supe.shape[0])

661
659


In [37]:
supe['period'] = 0
supe = supe.reset_index()
supe.head()

,date,minutesAsleep,deep,light,rem,wake,wake_time,in_bed_time (mins),period
0,2020-06-28,452.0,98,228,126,57,08:53:00,509,0
1,2020-06-27,434.0,76,289,69,72,08:38:30,506,0
2,2020-06-26,355.0,64,239,52,49,07:38:30,404,0
3,2020-06-25,441.0,53,298,90,76,08:44:30,517,0
4,2020-06-24,424.0,19,298,107,70,08:51:30,494,0


In [41]:
for d in supe.index:
    for i in per.index:
        if supe.iloc[d]['date'] >= per.iloc[i]['period_start_date'] and supe.iloc[d]['date'] <= per.iloc[i]['period_end_date']:
            supe.iloc[d]['period'] = 1

/home/xanadoo/.local/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [42]:
supe[supe['period']==1]
#supe.head()

,date,minutesAsleep,deep,light,rem,wake,wake_time,in_bed_time (mins),period
